In [2]:
import biom
import numpy as np
import pandas as pd
from biom import Table
from scipy.spatial import distance
from scripts.large_pickle import pickle_dump,pickle_load



In [5]:
"""
Import data and filter
by sum or reads/taxa

filter cutoffs
int, int
"""
n_sample = 0
n_feature = 0

# load data
bt = biom.load_table('../../data/ECAM-Qiita-10249/table.biom')
mf = pd.read_csv('../../data/ECAM-Qiita-10249/10249_20180418-081211.txt',sep='\t',index_col=0)
mf.index = mf.index.astype(str)

# drop subject with too few time points
mf = mf.replace('na',np.nan)
mf = mf.dropna(subset=['host_subject_id','month'])
mf = mf[mf.sampletype.isin(['Stool_Stabilizer'])]
mf['month'] = [int(x) for x in mf.month]
# too few samples at these months
mf = mf[mf.month.astype(int) <= 24]
mf = mf[~mf.month.isin([17,13,-1,21])]
month_subject = {k:sorted(set(map(int,df.month))) for k,df in mf.groupby('host_subject_id')}
keep_subjects = [k for k,v in month_subject.items() if len(v) > 0]
mf = mf[mf.host_subject_id.isin(keep_subjects)]

# filter table by sum cutoffs
keep_subject = bt.ids('sample')[bt.sum('sample') >= n_sample]
keep_subject = list(set(mf.index) & set(keep_subject))
bt.filter(keep_subject,axis='sample',inplace=True)

# filter taxa
keep_taxa = bt.ids('observation')[bt.sum('observation') > n_feature]
# drop and filter
bt.filter(keep_taxa,axis='observation',inplace=True)
# df
btdf = pd.DataFrame(bt.matrix_data.toarray(),
                    bt.ids('observation'),
                    bt.ids('sample'))
# ensure matched 
mf = mf.reindex(btdf.columns)

# save matched
# write mapping file to use for downstream
mf.index.name = '#SampleID'
mf.to_csv('../../data/ECAM-Qiita-10249/metadata-matched.tsv',sep='\t')
btdf.index.name = '#SampleID'
btdf.to_csv('../../data/ECAM-Qiita-10249/table-matched.tsv',sep='\t')
with biom.util.biom_open('../../data/ECAM-Qiita-10249/table-matched.biom', 'w') as f:
    bt.to_hdf5(f, "reference-hit-table-matched")
bt

3382 x 743 <class 'biom.table.Table'> with 47727 nonzero entries (1% dense)

In [7]:
len(set(mf.host_subject_id))

44

In [8]:
mf.delivery.value_counts()


Vaginal     444
Cesarean    299
Name: delivery, dtype: int64

In [9]:
from collections import Counter
Counter([list(set(df.delivery))[0] for i,df in mf.groupby('host_subject_id')])


Counter({'Vaginal': 24, 'Cesarean': 20})

In [10]:
set(mf.month)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 18, 19, 20, 22, 24}